Rule based trading to maximizing capital loss for tax savings while keeping the winnings


In [24]:
# Raw Package
import numpy as np
import pandas as pd
from datetime import date
from numpy.polynomial import Chebyshev
import os 

import pickle

#Data Source
import yfinance as yf

#Data viz
import plotly.graph_objs as go

In [25]:
#Interval by 1 day
#yf.pdr_override() #not needed
data = yf.download(tickers='SPY', period='1y', interval='1d')

#Print data
data

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2021-08-27,447.119995,450.649994,447.059998,450.250000,443.938110,77235100
2021-08-30,450.970001,453.070007,450.709991,452.230011,445.890350,48357400
2021-08-31,452.130005,452.489990,450.920013,451.559998,445.229736,59300200
2021-09-01,452.559998,453.109985,451.549988,451.799988,445.466400,48721400
2021-09-02,453.320007,454.049988,451.910004,453.190002,446.836884,42501000
...,...,...,...,...,...,...
2022-08-22,417.049988,417.230011,412.399994,413.350006,413.350006,77695600
2022-08-23,412.899994,415.420013,411.769989,412.350006,412.350006,49105200
2022-08-24,412.109985,415.109985,411.390015,413.670013,413.670013,49177800


In [26]:
fig = go.Figure()

#Candlestick
fig.add_trace(go.Candlestick (
    x=data.index,
    open=data['Open'],
    close=data['Close'],
    high=data['High'],
    low=data['Low'])
)

#Add titles
fig.update_layout (
    title='SPY shares',
    yaxis_title='Stock Price (USD)'
)

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=15, label='Month', step='month',stepmode='backward'),
            dict(count=1, label='Day', step='day',stepmode='backward'),
            dict(step='all'),
        ])
    )

)

fig.show()

Analysis

In [27]:
class inventory:
    cash = 100000
    capital_gain = 0
    capital_loss = 0
    holdings = pd.DataFrame(columns=['Ticker', 'Price', 'Amount'])



In [29]:
#Returns a number
def slope(df_subset): 
    #choosing a subset of the data
    max = df_subset.index.max()
    min = df_subset.index.min()

    day1 = data.at[min, "Close"]
    day3 = data.at[max, "Close"]
    numdays = len(df_subset)

    slopeval =(day3-day1)/numdays

    return slopeval


In [30]:
df_subset3 = data[date.today() - pd.offsets.Day(3):]
df_subset7 = data[date.today() - pd.offsets.Day(7):]
df_subset30 = data[date.today() - pd.offsets.Day(30):]

slope3 = slope(df_subset3)
slope7 = slope(df_subset7)
slope30 = slope(df_subset30)

print(slope3, slope7, slope30)

-7.100006103515625 -1.608001708984375 -0.3180948893229167


In [31]:
#Buy a stock, provide the ticker name and the # of stock to buy
def buy_stock(wallet, ticker, amount):
    
   data = yf.download(tickers=ticker, period='1d', interval='1d')

   closing_price = data["Close"].item()
   closing_price = round(closing_price, 2)

   if wallet.cash < closing_price*amount:
      raise Exception("Sorry not enough money. buy less.")

   purchase = {'Ticker': ticker, 'Price': closing_price, 'Amount': amount}

   #update balance
   wallet.holdings = wallet.holdings.append(purchase, ignore_index=True)
   wallet.cash = wallet.cash - closing_price*amount #current cash amount - purchased amount

   print("Holdings:")
   print(wallet.holdings)
   print("Cash on hand:", wallet.cash)

   

   


In [ ]:
#saving or loading previous wallet
def load():
    pickle_in = open('data.pickle', 'rb')
    wallet = pickle.load(pickle_in)
    print("loaded")

def save():
    pickle_out = open('data.pickle', 'wb')
    pickle.dump(wallet, pickle_out)
    pickle_out.close()
    print("saved. you may close.")

In [32]:

def sell_stock(wallet, sell_index, sell_amount):

    #wallet info: what you have
    ticker = wallet.holdings.at[sell_index, 'Ticker']
    amount = wallet.holdings.at[sell_index, 'Amount']
    price = wallet.holdings.at[sell_index, 'Price']


    #error check: enough stocks to sell
    if sell_amount > amount:
        raise Exception("Not enough to sell. sell less.")

    #current prices
    data = yf.download(tickers=ticker, period='1d', interval='1d')
    closing_price = data["Close"].item()
    closing_price = round(closing_price, 2)


    print(f"You have {amount} of {ticker} bought at ${price}, you're selling {sell_amount} for ${closing_price}")

    
    #Transaction - remove sold, convert to cash
    #update the amount you've sold 
    wallet.holdings.at[sell_index, 'Amount'] = amount - sell_amount 

    #update cash
    wallet.cash = wallet.cash + (sell_amount*closing_price)

    print(f"Updated cash balance: {wallet.cash}")
    print(f"Updated holdings: \n {wallet.holdings}")


Let's play! Use below functions to as example to start. 

In [40]:
#start the game by initiating the class
#wallet = inventory()

#check what you have
#wallet.holdings

#buy stock
buy_stock(wallet, 'AAPL', 150)
buy_stock(wallet, 'SPY', 100)
buy_stock(wallet, 'AAPL', 25)

#sell stock
#sell_stock(wallet, 0, 15)
#sell_stock(wallet, 0, 5)


[*********************100%***********************]  1 of 1 completed
Holdings:
  Ticker   Price Amount
0    SPY  405.31     15
1   AAPL  163.62    150
Cash on hand: 69377.35


C:\Users\sunny\AppData\Local\Temp\ipykernel_14296\2044263289.py:14: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



69377.35
